# Zero Knowledge Week 1


## Modular arithmetic essentials
### Benchmarks
Here is how you do modular arithmetic in python. Run the two cells below and note how different their execution times are. You should use `pow` instead of doing modular arithmetic with the elementary operators.

In [3]:
import time

s = time.time()
ans = 1002583 ** 939001 % 2003951
t = time.time()

print("calculating {} took {:.2f} seconds".format(ans, t - s))

calculating 1590868 took 3.70 seconds


In [4]:
s = time.time()
ans = pow(1002583,939001,2003951)
t = time.time()

print("calculating {} operation took {:.2f} seconds".format(ans, t - s))

calculating 1590868 operation took 0.00 seconds


### Encrypting a number

If we start with a generator `g` and raise it to a power `s` mod `n`, then in general we cannot determine `s` even if `g` and `n` are known. This is the discrete logarithm problem.

We can say we "encrypt" `s` using the scheme

$$ E(s) = g^{s} \pmod n $$

But this means we need to choose `g` and `n` carefully. If `g` is 3, `s` is 2, and `n` is a large number, we can intelligently guess what `s` is in this case. We want to pick `g` and `n` such that all members "wrap around." We won't concern ourselves with this here, but you should be aware of it. The other issue is not picking a sufficiently large `n`. 

### Computing the discrete logarithm
If we do `s_p = g^s mod n`, and make `g` and `n` public, then it is infeasible to compute `s`. But this is only true if `n` is sufficiently large.

In the following example, we provide `s_p`, `g`, and `n`. Use python to bruteforce the solution to find `s`.

$$g^{ac} = (g^{a})^{c} \pmod p$$

In [5]:
n = 9551
g = 5
encrypted_number = 5666

<font color='red'>**Assignment 1: Use python to bruteforce the solution to find `s`**</font>

In [6]:
def brute_force(n,g,encrypted_number): 
    for i in range(0,10000,1): 
        if pow(g,i,n) == encrypted_number: 
            return i 


student_solution = brute_force(9551,5,5666)

def find_s_brute_force(g, s_p, n):
    """
    Finds the discrete logarithm s for given g, s_p, and n by brute force.
    It solves for s in the equation g^s mod n = s_p.
    
    Parameters:
    g (int): The base of the exponentiation.
    s_p (int): The result of the modular exponentiation.
    n (int): The modulus for the exponentiation.
    
    Returns:
    int: The exponent s, if found. Otherwise, returns None.
    """
    for s in range(n):
        if pow(g, s, n) == s_p:
            return s  # Found the correct s
    return None  # s not found within the range [0, n)

# Given values
g = 3
s_p = 8
n = 17

# Finding s
s = find_s_brute_force(g, s_p, n)

# Displaying the result
if s is not None:
    print(f"The value of s such that {g}^s mod {n} = {s_p} is s = {s}")
else:
    print("Could not find a value of s that satisfies the equation with the given parameters.")


The value of s such that 3^s mod 17 = 8 is s = 10


In [7]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 2531


### Computing the discrete logarithm the smart way
The smart way to do optimal algorithms is to [copy and paste from stackoverflow](https://stackoverflow.com/a/58665206/2079806). You do not have to understand how the library works

<font color='red'>**Assignment 2: Break the cryptography below and obtain the secret number**</font>

In [8]:
n = 1000004119
g = 5
encrypted_number = 767805982

In [9]:
from sympy.ntheory import discrete_log

# Given values
n = 1000004119
g = 5
encrypted_number = 767805982

# Using the discrete_log function to find the exponent
# that satisfies g^s mod n = encrypted_number
student_solution = discrete_log(n, encrypted_number, g)

# Verification to ensure the solution is correct
assert pow(g, student_solution, n) == encrypted_number

# Print the solution
print("The secret number (s) is:", student_solution)

student_solution is 420


## How Zero Knowledge Addition Works

A Zero Knowledge Proof verifies a computation is true without revealing a the inputs to the computation.

That is, if a + b = c, then we want a "trapdoor function" E(x) such that E(a)■E(b) = E(c) where ■ is the binary operator of the group E(x) is an element of.

So to prove a + b = c without revealing a, b, or c, we apply A = E(a), B = E(b), C = E(c) and give (A, B, C) to a verifier.

The verifier will then check A ■ B == B.

Although they don't know the values A, B, and C "came from", they do know the unencrypted values form a valid sum.

## Zero Knowledge Addition
The following property is very important. It lets us verify the addition of numbers (integers under addition) using another group: $g^x \pmod p$ under multiplication. Because of the discrete logarithm, we cannot easily go from $g^x \pmod p$ to recover x.

a + b = c is homomorphic to $g^ag^b = g^c \pmod p$.

$$ g^{a}g^{b} = g^{a + b} \pmod n $$

Try a few different values of `a` and `b` yourself to see it in action

In [11]:
a = 3
b = 334
print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))

False


Wait what?! That's supposed to be true?

If the expression pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n) reads False, it's likely due to an oversight in how the multiplication is handled with respect to modulo 
�
n. Specifically, when you multiply pow(g, a, n) by pow(g, b, n), you need to take the result modulo 
�
n again to ensure it's correctly compared to pow(g, a + b, n).



<font color='red'>**Assignment 3: The code above has a bug. What is the bug?**</font>

a = 3
b = 334
g = 5
n = 1000004119

# Correctly applying modulo n after multiplication
result_corrected = (pow(g, a, n) * pow(g, b, n)) % n == pow(g, a + b, n)

print(result_corrected)

In [12]:
print(((g**a) * (g**b) % n) == pow(g, a + b, n))

True


## Zero Knowledge Subtraction
We can also encrypt the operation a - b. This is the same as 

$$ g^{a}g^{-b} = g^{a-b} $$

But we can't just stick a negative sign in front of the exponent

In [13]:
g = 2
n = 3
print(g ** -5 % n)

0.03125


The above is not an integer! However, python is smart enough to know what you are doing if you use `pow`. To test this, we expect

$$ g^{-a}g^{a} = 1 $$

because g^0 is 1, and `a - a = 0`.

In [14]:
a = 22
pow(g, a, n) * pow(g, -a, n) % n == 1

True

So what magic is happening behind the scenes? The formula below is used to compute the modular inverse.

$$ a^{-1} = a^{n - 2} \pmod n $$

In [15]:
a_inv = a ** (n - 2) % n
a_inv * a % n == 1

True

## Multiplication by a constant
Multiplication by a constant is really just repeated addition

$$ (g^{a})^{4} = g^{a} g^{a} g^{a} g^{a} $$

This allows us to "multiply by a constant" in a zero knowledge fashion.

In [16]:
a = 15
pow(g, a, n) * pow(g, a, n) * pow(g, a, n) * pow(g, a, n) % n == pow(g, a * 4, n)

True

Of course, it would be annoying to multiply like that if you have a big coefficient, so the following is better

In [17]:
pow(pow(g, a, n), 4 , n) == pow(g, a * 4, n)

True

## I know the solution to a systems of equations

Now you should be able to prove to me that you know the solution to in a zero knowledge fashion

$$
2x + 8y = 7944\\
5x + 3y = 4764
$$

<font color='red'>**Assignment 4: convert the above equation to a zk proof system where you can prove to a verifier (with an agreed upon g and n) that you know the solution to an agreed upon system of equation, without revealing the solution**</font>



In [2]:
from sympy import symbols, Eq, solve

# Define symbols
x, y = symbols('x y')

# Solve the system of equations
solution = solve((Eq(2*x + 8*y, 7944), Eq(5*x + 3*y, 4764)), (x, y))

# Given parameters (in a real scenario, n should be a large prime for security)
g = 2
n = 101

# "Encrypt" the solutions by explicitly converting them to int
x_encrypted = pow(g, int(solution[x]), n)
y_encrypted = pow(g, int(solution[y]), n)

print("Encrypted x:", x_encrypted)
print("Encrypted y:", y_encrypted)



Encrypted x: 95
Encrypted y: 92


## Convert a rational number to a finite field element

<font color='red'>**Assignment 5: Compute 53/192 + 61/511 (mod 1033) in python. Show your work. You are not required to implement euclid's algorithm. Show it is equal to the original rational number sum: https://www.wolframalpha.com/input?i=53%2F192+%2B+61%2F511**</font>

In [5]:
# Function to compute the modular multiplicative inverse
def modinv(a, modulus):
    # Extended Euclidean Algorithm for finding the modular inverse
    g, x, y = extended_gcd(a, modulus)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % modulus

# Extended Euclidean Algorithm
def extended_gcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = extended_gcd(b % a, a)
        return (g, x - (b // a) * y, y)

# Given values
num1, den1 = 53, 192
num2, den2 = 61, 511
modulus = 1033

# Compute the modular multiplicative inverse of the denominators
den1_inv_mod = modinv(den1, modulus)
den2_inv_mod = modinv(den2, modulus)

# Compute modular sum of the fractions
mod_sum = (num1 * den1_inv_mod + num2 * den2_inv_mod) % modulus

# Calculate the original sum and its modular equivalent for comparison
original_sum_fraction = (num1 / den1 + num2 / den2)
original_sum_mod = (original_sum_fraction * modulus) % modulus

print(f"Modular sum of the fractions: {mod_sum}")
print(f"Modular equivalent of the original sum: {original_sum_mod}")



Modular sum of the fractions: 514
Modular equivalent of the original sum: 408.46415321265494


In [18]:
x = 420
y = 888
print(pow(pow(2,x,11),2,11) * pow(pow(2,y,11),8,11) % 11 == pow(2,7944,11) and pow(pow(2,x,11),5,11) * pow(pow(2,y,11),3,11) % 11 == pow(2,4764,11))

True


In [19]:

numerator = 53*511 + 61*192
denominator = 511 * 192

numerator_mod = pow(numerator,1,1033)
numerator_mod_inverse = pow(numerator,-1,1033)

denominator_mod = pow(denominator,1,1033) 
denominator_mod_inverse = pow(denominator,-1,1033) 

finite_field_element = numerator_mod * denominator_mod_inverse % 1033


print(denominator_mod_inverse == (finite_field_element * numerator_mod_inverse % 1033) and numerator_mod == (finite_field_element * denominator_mod % 1033))


True
